In [1]:
import os
import pandas as pd
import numpy as np
import joblib
from src.models.predict_model import PredictModel

#### Prep Data

In [2]:
prepared_train_data = pd.read_csv("../data/processed/series_train_preprocessed.csv")
prepared_test_data = pd.read_csv("../data/processed/series_test_preprocessed.csv")
prepared_validation_data = pd.read_csv("../data/processed/series_validate_preprocessed.csv")

In [3]:
X_train = prepared_train_data.drop('target', axis=1)
y_train = prepared_train_data['target']

X_test = prepared_test_data.drop('target', axis=1)
y_test = prepared_test_data['target']

X_val = prepared_validation_data.drop('target', axis=1)
y_val = prepared_validation_data['target']

top_20_features_final = joblib.load("../artefacts/top_20_features_final.pkl")

X_train_20_final = X_train[top_20_features_final]
X_test_20_final = X_test[top_20_features_final]
X_val_20_final = X_val[top_20_features_final]

# reshape the data
def reshape_data(X_train, X_test, X_val):
    timesteps = 1
    X_train = X_train.values.reshape(-1, timesteps, len(X_train.columns))
    X_test = X_test.values.reshape(-1, timesteps, len(X_test.columns))
    X_val = X_val.values.reshape(-1, timesteps, len(X_val.columns))
    return X_train, X_test, X_val

X_train_20_final_gru, X_test_20_final_gru, X_val_20_final_gru = reshape_data(
    X_train_20_final,
    X_test_20_final,
    X_val_20_final)

In [4]:
os.chdir('..')

In [5]:
X_test_20_final.loc[[0],:]

,high_t,tema_14_t,wcl_price_t,med_price_t-2,med_price_t,close_t,typ_price_t,avg_price_t,tema_9_t,open_t,typ_price_t-1,wcl_price_t-2,dema_10_t-1,close_t-1,upperband_5_t,tema_14_t-1,middleband_5_t,ma_5_t,med_price_t-1,tema_9_t-1
0,0.227233,0.228873,0.229047,0.229325,0.229235,0.229233,0.228985,0.229024,0.22871,0.229556,0.228962,0.229329,0.228957,0.229616,0.225662,0.229066,0.228298,0.228298,0.229008,0.228929


#### Backtesting

In [6]:
stacker = PredictModel()


c:\Users\ab6ri\OneDrive\Documents\Personal_Projects\simmer_bot\.venv\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.6.0 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


1/1 [==============================] - 3s 3s/step


1.0563871354244376

In [7]:
stacker.stack_predict(pd.DataFrame(X_test_20_final.loc[[2],:]))

1/1 [==============================] - 0s 26ms/step


1.0562469265871743